## Milvus Vector DB for RAG

### Create a collection of legal docs

In [20]:
%pip install pymilvus

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [21]:
from pymilvus import connections

# If using Docker standalone Milvus
connections.connect("default", host="127.0.0.1", port="19530")

In [22]:
from pymilvus import db
from pymilvus import Collection, FieldSchema, CollectionSchema, DataType

# 1. Create a new database
db.create_database("rag_db")

# 2. Switch to that database
db.using_database("rag_db")

# ----- Create schema -----
fields = [
    FieldSchema("doc_id", DataType.INT64, is_primary=True, auto_id=False),
    FieldSchema("title", DataType.VARCHAR, max_length=200),
    FieldSchema("domain", DataType.VARCHAR, max_length=100),
    FieldSchema("content", DataType.VARCHAR, max_length=2000),
    FieldSchema("embedding", DataType.FLOAT_VECTOR, dim=384) 
]

schema = CollectionSchema(fields, description="Policy documents with embeddings")
collection = Collection("policy_docs_collection", schema)

# ----- Create index -----
index_params = {
    "index_type": "IVF_FLAT",
    "metric_type": "COSINE",
    "params": {"nlist": 128},
}
collection.create_index(field_name="embedding", index_params=index_params)

2025-11-13 14:00:04,609 [ERROR][handler]: RPC error: [create_database], <MilvusException: (code=65535, message=database already exist: rag_db)>, <Time:{'RPC start': '2025-11-13 14:00:04.602382', 'RPC error': '2025-11-13 14:00:04.608980'}> (decorators.py:253)


MilvusException: <MilvusException: (code=65535, message=database already exist: rag_db)>

In [ ]:
%pip install -q sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# ----- Example data -----
content_chunks = [
    {
        "doc_id": 1,
        "section": "Pay Policies",
        "title": "Employee Pay Policy",
        "domain": "Human Resources",
        "content": "Employees are paid bi-weekly via direct deposit."
    },
    {
        "doc_id": 1,
        "section": "Leave of Absence",
        "title": "Leave Request and Approval Process",
        "domain": "Human Resources",
        "content": "Employees must submit a leave request for approval."
    },
    {
        "doc_id": 1,
        "section": "Internet Use",
        "title": "Acceptable Use of Company Internet",
        "domain": "IT & Security",
        "content": "Company internet must be used for work-related tasks only."
    },
    {
        "doc_id": 2,
        "section": "Break at Work",
        "title": "Employee Break Policy",
        "domain": "Workplace Operations",
        "content": "Employees can take an hour break."
    },
    {
        "doc_id": 2,
        "section": "Harassment",
        "title": "Workplace Harassment Policy",
        "domain": "Compliance",
        "content": "Interact with each employee with respect."
    }
]


# content_chunks_list = []
# for chunk in content_chunks:
#     content_chunks_list.append(chunk["content"])
content_chunks_list = [chunk["content"] for chunk in content_chunks]
print(content_chunks_list)

from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-MiniLM-L6-v2")

doc_vectors = model.encode(content_chunks_list)
doc_vectors.shape

['Employees are paid bi-weekly via direct deposit.', 'Employees must submit a leave request for approval.', 'Company internet must be used for work-related tasks only.', 'Employees can take an hour break.', 'Interact with each employee with respect.']


/Users/suma/Training/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(5, 384)

In [ ]:
# ---- Build columnar data ----
doc_ids = [int(i + 1) for i in range(len(content_chunks))]             # INT64
titles = [str(doc["title"]) for doc in content_chunks]                 # VARCHAR
domains = [str(doc["domain"]) for doc in content_chunks]               # VARCHAR
content = [str(doc["content"]) for doc in content_chunks]               # VARCHAR
embeddings = [list(map(float, vec)) for vec in doc_vectors]       # FLOAT_VECTOR(768)


# ---- Insert column-wise ----
collection.insert([doc_ids, titles, domains, content, embeddings])
collection.flush()

print(f"Successfully inserted {len(doc_ids)} documents into Milvus.")

Successfully inserted 5 documents into Milvus.


In [ ]:
#Load the collection before searching or querying
collection.load()
res = collection.query(expr="doc_id > 0", output_fields=["doc_id", "title", "domain", "content", "embedding"], limit=5)
print(res)

data: ["{'title': 'Employee Pay Policy', 'domain': 'Human Resources', 'content': 'Employees are paid bi-weekly via direct deposit.', 'embedding': [0.024725137278437614, -0.009081455878913403, 0.038871295750141144, 0.03230151906609535, -0.058905523270368576, 0.03912458196282387, 0.026017971336841583, -0.043690890073776245, -0.019160090014338493, -1.1300465985186747e-06, 0.013653510250151157, 0.004326691851019859, -0.05378556624054909, 0.006868904456496239, 0.007638247683644295, -0.06069640815258026, -0.037602201104164124, 0.007140938192605972, 0.13387173414230347, 0.0017353893490508199, -0.00307556614279747, -0.07781298458576202, -0.08451440185308456, -0.003216360229998827, 0.1338280886411667, -0.03643585368990898, 0.01692275144159794, 0.0486147478222847, -0.04028798267245293, 0.019676750525832176, -0.04371284693479538, 0.05151141807436943, -0.016299592331051826, -0.039059221744537354, -0.004453878849744797, 0.0004744756151922047, -0.03120509535074234, 0.04895618557929993, 0.01283164881

In [ ]:
# print(utility.has_collection("demo_collection"))

# # Get details about a specific collection
# # Get collection details
# collection = Collection("demo_collection")  # instantiate the collection object
print(collection.schema)                    # show the schema
print(collection.num_entities)              # number of entities
print(collection.description)               # optional

{'auto_id': False, 'description': 'Policy documents with embeddings', 'fields': [{'name': 'doc_id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': False}, {'name': 'title', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 200}}, {'name': 'domain', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 100}}, {'name': 'content', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 2000}}, {'name': 'embedding', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 384}}], 'enable_dynamic_field': False}
5
Policy documents with embeddings


In [ ]:
# Display results
for record in res:
    print(f"Doc ID: {record['doc_id']}")
    print(f"Title: {record['title']}")
    print(f"Domain: {record['domain']}")
    print(f"Content: {record['content']}")
    # Show only first 5 embedding values for readability
    print(f"Embedding (first 5): {record['embedding'][:5]}")
    print("-" * 80)

Doc ID: 1
Title: Employee Pay Policy
Domain: Human Resources
Content: Employees are paid bi-weekly via direct deposit.
Embedding (first 5): [0.024725137278437614, -0.009081455878913403, 0.038871295750141144, 0.03230151906609535, -0.058905523270368576]
--------------------------------------------------------------------------------
Doc ID: 2
Title: Leave Request and Approval Process
Domain: Human Resources
Content: Employees must submit a leave request for approval.
Embedding (first 5): [0.03315506502985954, 0.048533786088228226, 0.04736274480819702, 0.02527041547000408, 0.05931208282709122]
--------------------------------------------------------------------------------
Doc ID: 3
Title: Acceptable Use of Company Internet
Domain: IT & Security
Content: Company internet must be used for work-related tasks only.
Embedding (first 5): [-0.0713590756058693, -0.03066469170153141, 0.031837716698646545, -0.07750097662210464, -0.00503249978646636]
------------------------------------------------

In [ ]:
query = "What’s the leave policy?"
query_vector = model.encode([query])[0]
query_vector[:5]  # Show only first 5 values

array([0.0604582 , 0.03882856, 0.01172254, 0.03124589, 0.12124222],
      dtype=float32)

In [ ]:
# Search for closest match only in the 'Human Resources' domain
results = collection.search(
    data=[query_vector],
    anns_field="embedding",
    param={"metric_type": "COSINE", "params": {"nprobe": 10}},
    limit=3,
    expr='domain == "Human Resources"',
    output_fields=["doc_id", "title", "domain", "content"]
)

context_sring = ""
for res in results[0]:
    print(f"doc_id={res.entity.get('doc_id')}, "
          f"title={res.entity.get('title')}, "
          f"domain={res.entity.get('domain')}, "
          f"content={res.entity.get('content')}, "
          f"score={res.distance}")
    context_sring += f"\n -- \n {res.entity.get('content')} " # Append content to context string

print("\nContext String for RAG:\n", context_sring)    

doc_id=2, title=Leave Request and Approval Process, domain=Human Resources, content=Employees must submit a leave request for approval., score=0.61988365650177
doc_id=1, title=Employee Pay Policy, domain=Human Resources, content=Employees are paid bi-weekly via direct deposit., score=0.23205099999904633

Context String for RAG:
 
 -- 
 Employees must submit a leave request for approval. 
 -- 
 Employees are paid bi-weekly via direct deposit. 


In [23]:
from llm_utility import ask_question_open_ai 

query = "What’s the leave policy?"
response = ask_question_open_ai(query, context_sring)
response

'According to the provided context, the leave policy requires employees to submit a leave request for approval. (No other details about leave types, durations, or timelines are given.)'

In [24]:
print(f"User query: {query}")
print(f"Context: {context_sring}")

print(f"\n\nOpen AI Response: {response}")

User query: What’s the leave policy?
Context: 
 -- 
 Employees must submit a leave request for approval. 
 -- 
 Employees are paid bi-weekly via direct deposit. 


Open AI Response: According to the provided context, the leave policy requires employees to submit a leave request for approval. (No other details about leave types, durations, or timelines are given.)
